In [28]:
from dataclasses import dataclass
from pathlib import Path 

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path 
    tokenizer_name: Path
    

In [4]:
import os

In [2]:
%pwd

'd:\\MLOPS_TextSummarizer\\research'

In [5]:
os.chdir("../")
%pwd

'd:\\MLOPS_TextSummarizer'

In [6]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories


In [24]:
class ConfigurationManager:

    def __init__(self, config_path=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )

        return data_transformation_config


In [25]:
import os 
from src.textSummarizer.logging import logger 
from transformers import AutoTokenizer 
from datasets import load_from_disk


### Data Transformation Component

In [ ]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch): 
        input_encodings = self.tokenizer(
            example_batch["dialogue"],
            max_length=1024,
            truncation=True,
            padding="max_length"
        )

        target_encodings = self.tokenizer(
            text_target=example_batch["summary"],  
            max_length=128,
            truncation=True,
            padding="max_length"
        )

        return {
            "input_ids": input_encodings["input_ids"],
            "attention_mask": input_encodings["attention_mask"],
            "labels": target_encodings["input_ids"]
        }

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))

In [29]:
config = ConfigurationManager()
data_transformation_config = config.get_data_transformation_config()
data_transformation = DataTransformation(config=data_transformation_config)
data_transformation.convert()


[2025-06-29 15:35:17,476: INFO: common: yaml file :config\config.yaml loaded_successfully]
[2025-06-29 15:35:17,480: INFO: common: yaml file :params.yaml loaded_successfully]
[2025-06-29 15:35:17,480: INFO: common: created direactory as :artifacts]
[2025-06-29 15:35:17,493: INFO: common: created direactory as :artifacts/data_transformation]


d:\MLOPS_TextSummarizer\venv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Vansh\.cache\huggingface\hub\models--google--pegasus-cnn_dailymail. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 24459.37 examples/s]
